In [1]:
from glob import glob #'glob' is used to search for a file with a specific file name in a csv or text file format.
import os #'os' is for interacting with the operating system.
import mne #'mne' is an open-source Python module for processing, analysis, and visualization of functional neuroimaging data (EEG, MEG, sEEG,etc...).
import numpy as np #numpy to deal with mathematical functions on arrays
import pandas as pd #pandas to perform various functions on dataframes
import matplotlib.pyplot
import scipy # we us SciPy to help us interact with the data at a more complex level.
from scipy import io

In [2]:
# Here we are just importing our dataset.
TDC_data_path=r'C:\Users\Aadithya\OneDrive\Desktop\Data\CleanData\CleanData_TDC\Rest'
IDD_data_path=r'C:\Users\Aadithya\OneDrive\Desktop\Data\CleanData\CleanData_IDD\Rest'
#IDD_data_path=os.path.abspath

In [3]:
#In this step we define a function to connvert our given matlab files to mne files so that we can easily use them in our model.
def convertmat2mne(data):
    ch_names = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4'] # Here we specify the channels and their type.
    ch_types = ['eeg'] * 14
    sampling_freq=128
    info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
    info.set_montage('standard_1020') # Here we specify the location of the elctrodes.
    data=mne.io.RawArray(data, info)
    data.set_eeg_reference()
    data.filter(l_freq=1,h_freq=30)
    epochs=mne.make_fixed_length_epochs(data,duration=4,overlap=0)
    return epochs.get_data() # As final output we now have data in a 3d format.

In [7]:
%%capture
# Here we convert our data.
idd_subject=[]
for idd in glob(IDD_data_path + '/*.mat'):
    data=scipy.io.loadmat(idd)['clean_data']
    data=convertmat2mne(data)
    idd_subject.append(data)

In [8]:
%%capture
# Same as above.
tdc_subject=[]
for tdc in glob(TDC_data_path+'/*.mat'):
    data=scipy.io.loadmat(tdc)['clean_data']
    data=convertmat2mne(data)
    tdc_subject.append(data)

In [9]:
# In this step we are finding out the length of each dataset to be evaluated.
control_epochs_labels=[len(i)*[0] for i in tdc_subject]
patients_epochs_labels=[len(i)*[1] for i in idd_subject]
print(len(control_epochs_labels),len(patients_epochs_labels))

7 7


In [10]:
# Here we combine our two data sets to create one singular datatset.
data_list=tdc_subject+idd_subject
label_list=control_epochs_labels+patients_epochs_labels
groups_list=[[i]*len(j) for i, j in enumerate(data_list)]
print(len(data_list),len(label_list),len(groups_list))

14 14 14


In [11]:
# 
from sklearn.model_selection import GroupKFold,LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold()
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.preprocessing import StandardScaler
#https://stackoverflow.com/questions/50125844/how-to-standard-scale-a-3d-matrix
class StandardScaler3D(BaseEstimator,TransformerMixin):
    #batch, sequence, channels
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self,X,y=None):
        self.scaler.fit(X.reshape(-1, X.shape[2]))
        return self

    def transform(self,X):
        return self.scaler.transform(X.reshape( -1,X.shape[2])).reshape(X.shape)

In [12]:
import numpy as np
data_array=np.concatenate(data_list)
label_array=np.concatenate(label_list)
group_array=np.concatenate(groups_list)
data_array=np.moveaxis(data_array,1,2)

print(data_array.shape,label_array.shape,group_array.shape)

(420, 512, 14) (420,) (420,)


In [13]:
accuracy=[]
for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler3D()
    train_features=scaler.fit_transform(train_features)
    val_features=scaler.transform(val_features)
   

    break

In [14]:
train_features.shape

(330, 512, 14)

In [15]:
from tensorflow.keras.layers import Input,Dense,concatenate,Flatten,GRU,Conv1D
from tensorflow.keras.models import Model

In [16]:
def block(input):
    conv1 = Conv1D(32, 2, strides=2,activation='relu',padding="same")(input)
    conv2 = Conv1D(32, 4, strides=2,activation='relu',padding="causal")(input)
    conv3 = Conv1D(32, 8, strides=2,activation='relu',padding="causal")(input)
    x = concatenate([conv1,conv2,conv3],axis=2)
    return x

In [17]:
input= Input(shape=(512,14))
block1=block(input)
block2=block(block1)
block3=block(block2)

In [18]:
gru_out1 = GRU(32,activation='relu',return_sequences=True)(block3)
gru_out2 = GRU(32,activation='relu',return_sequences=True)(gru_out1)
gru_out = concatenate([gru_out1,gru_out2],axis=2)
gru_out3 = GRU(32,activation='relu',return_sequences=True)(gru_out)
gru_out = concatenate([gru_out1,gru_out2,gru_out3])
gru_out4 = GRU(32,activation='relu')(gru_out)

In [19]:
predictions = Dense(1,activation='sigmoid')(gru_out4)
model = Model(inputs=input, outputs=predictions)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [21]:
model.fit(train_features,train_labels,epochs=20,batch_size=128,validation_data=(val_features,val_labels))

Epoch 1/20
3/3 [==============================] - 1s 510ms/step - loss: 1.3079e-07 - accuracy: 1.0000 - val_loss: 0.0252 - val_accuracy: 0.9889
Epoch 2/20
3/3 [==============================] - 1s 446ms/step - loss: 1.3011e-07 - accuracy: 1.0000 - val_loss: 0.0254 - val_accuracy: 0.9889
Epoch 3/20
3/3 [==============================] - 1s 431ms/step - loss: 1.2934e-07 - accuracy: 1.0000 - val_loss: 0.0256 - val_accuracy: 0.9889
Epoch 4/20
3/3 [==============================] - 1s 439ms/step - loss: 1.2849e-07 - accuracy: 1.0000 - val_loss: 0.0257 - val_accuracy: 0.9889
Epoch 5/20
3/3 [==============================] - 1s 436ms/step - loss: 1.2760e-07 - accuracy: 1.0000 - val_loss: 0.0259 - val_accuracy: 0.9889
Epoch 6/20
3/3 [==============================] - 1s 456ms/step - loss: 1.2689e-07 - accuracy: 1.0000 - val_loss: 0.0261 - val_accuracy: 0.9889
Epoch 7/20
3/3 [==============================] - 1s 459ms/step - loss: 1.2606e-07 - accuracy: 1.0000 - val_loss: 0.0262 - val_accuracy: